In [1]:
from mosqlient import get_infodengue
import pandas as pd
import numpy as np
from datetime import date
import time
from aiohttp import ClientConnectionError

ufs = [
    "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", 
    "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", 
    "RS", "RO", "RR", "SC", "SP", "SE", "TO"
]
years = range(2024, 2025)
arboviroses = ['dengue', 'chikungunya'] 
dfs = []

In [ ]:
max_retries = 5
retry_delay = 120 

for arbovirose in arboviroses:
    for uf in ufs:

        if uf != ufs[0]:
            print('Esperando 5:30min')
            time.sleep(retry_delay)

        for year in years:

            print(f'{uf} - {arbovirose} - {str(year)}')

            start_date = f'{year}-01-01'
            if year == date.today().year:
                end_date = str(date.today())
            else:
                end_date = f'{year}-12-31'

            attempt = 0
            while attempt < max_retries:
                try:
                    # Tenta fazer a requisição
                    df_year = get_infodengue(
                        start_date=start_date,
                        end_date=end_date,
                        disease=arbovirose,
                        uf=uf
                    )
                    df_year['uf'] = uf
                    df_year['CID'] = arbovirose

                    dfs.append(df_year)
                    
                    time.sleep(300*0.1)
                    print('Próximo ano')
                    break 

                except ClientConnectionError as e:
                    print(f"Erro de conexão: {e}. Tentando novamente em 5 minutos.")
                    attempt += 1
                    if attempt < max_retries:
                        time.sleep(retry_delay) 
                    else:
                        print("Número máximo de tentativas atingido. Pulando para o próximo ano.")

                except Exception as e:
                    print(f"Ocorreu um erro: {e}. Tentando novamente em 5 minutos.")
                    attempt += 1
                    if attempt < max_retries:
                        time.sleep(retry_delay)  
                    else:
                        print("Número máximo de tentativas atingido. Pulando para o próximo ano.")

        # Salvando ao finalizar cada estado
        if dfs:
            df_uf = pd.concat(dfs, ignore_index=True)
            df_uf.to_csv('../data/df_mosqlimate_uf_chik.csv', sep=";", decimal=".", index=False)

# Salvando ao finalizar tudo
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv('../data/df_mosqlimate_chik.csv', sep=";", decimal=".", index=False)

In [5]:
len(df_final['uf'].unique())

27

In [6]:
pd.crosstab(df_final['uf'].unique(), df_final['CID'])

ValueError: All arrays must be of the same length